# Week 2 Lab
## Machine Learning System (MLS) Design

### Loading CSV data

In [22]:
#Importing required packages 

#import seaborn as sns
#import pandas as pd

In [19]:
df = pd.read_csv('Sales Transaction v.4a.csv', index_col='TransactionNo')

### Looking at data details

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536350 entries, 581482 to C536548
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Date         536350 non-null  object 
 1   ProductNo    536350 non-null  object 
 2   ProductName  536350 non-null  object 
 3   Price        536350 non-null  float64
 4   Quantity     536350 non-null  int64  
 5   CustomerNo   536295 non-null  float64
 6   Country      536350 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 32.7+ MB


In [9]:
df.describe(include = 'all')

,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
count,536350,536350,536350,536350.000000,536350.000000,536295.000000,536350
unique,305,3768,3768,NaN,NaN,NaN,38
top,12/5/2019,85123A,Cream Hanging Heart T-Light Holder,NaN,NaN,NaN,United Kingdom
freq,5299,2378,2378,NaN,NaN,NaN,485095
mean,NaN,NaN,NaN,12.662182,9.919347,15227.893178,NaN
std,NaN,NaN,NaN,8.490450,216.662300,1716.582932,NaN
min,NaN,NaN,NaN,5.130000,-80995.000000,12004.000000,NaN
25%,NaN,NaN,NaN,10.990000,1.000000,13807.000000,NaN
50%,NaN,NaN,NaN,11.940000,3.000000,15152.000000,NaN
75%,NaN,NaN,NaN,14.090000,10.000000,16729.000000,NaN


In [11]:
df.head(10)

,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
TransactionNo,,,,,,,
581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom
581475,12/9/2019,21705,Bag 500g Swirly Marbles,10.65,24,13069.0,United Kingdom
581475,12/9/2019,22118,Joy Wooden Block Letters,11.53,18,13069.0,United Kingdom
581475,12/9/2019,22119,Peace Wooden Block Letters,12.25,12,13069.0,United Kingdom
581475,12/9/2019,22217,T-Light Holder Hanging Lace,10.65,12,13069.0,United Kingdom


### Insights Based on the Dataset

`Expected Structure:`

Based on an initial understanding, the dataset was expected to have __8 columns__, including:

- 5 text-based columns: TransactionNo, Date, ProductNo, ProductName, and Country.
- 3 numeric columns: Price, Quantity, and CustomerNo.

Among the text-based columns, Country can be categorized as categorical variables since they have a finite set of possible values.

`Actual Dataset Analysis:`

__Number of Entries:__ The dataset contains 536,350 rows of transaction data.

__Missing Values:__ There are some missing values, specifically in the CustomerNo column.

__Consistency with Expectations:__ The overall structure of the dataset aligns with what was expected (text-based and numeric columns).

`Range of Values:`

__Text-Based Columns:__
- TransactionNo and ProductNo contain unique identifiers, and Country has distinct categories (e.g., "United Kingdom").

__Numeric Columns:__
- Price and Quantity have a meaningful range, with Price being continuous and Quantity being discrete.
- CustomerNo is a numeric ID but has missing values.

`Summary:`

- The dataset is well-structured and matches expectations, making it suitable for further analysis.
- Some preprocessing will be needed to address missing values in CustomerNo.
- The range of values across columns ensures that there is sufficient variation for meaningful analysis.

### Importing data to Database

In [36]:
# Loading db package
# from sqlalchemy import create_engine

#### Connection to DB

In [24]:
df.shape

(536350, 7)

In [92]:
host = r'127.0.0.1'
db = r'MSDS610'
user = r'postgres'
pw = r'Pa55w0rd'
port = r'5432'
schema = r'raw'

In [94]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [96]:
sql="select tables.table_name from information_schema.tables where (table_schema ='"+schema+"')order by 1;"
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name


Connection Succesful

#### Loading Data

In [98]:
table_name = r'sales_transaction'

This is where we insert or csv data into our database table

In [100]:
df.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

536350

In [102]:
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name
0,sales_transaction


Table created successfully. 

#### Retrieving Data

In [104]:
sql=r'SELECT * FROM ' + schema + '.' + table_name
df_check = pd.read_sql(sql, db_conn, index_col=None)

In [106]:
df_check.head(10)

,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom
5,12/9/2019,21705,Bag 500g Swirly Marbles,10.65,24,13069.0,United Kingdom
6,12/9/2019,22118,Joy Wooden Block Letters,11.53,18,13069.0,United Kingdom
7,12/9/2019,22119,Peace Wooden Block Letters,12.25,12,13069.0,United Kingdom
8,12/9/2019,22217,T-Light Holder Hanging Lace,10.65,12,13069.0,United Kingdom
9,12/9/2019,22216,T-Light Holder White Lace,10.55,24,13069.0,United Kingdom


Data loaded to Postgres successfully.